# VanMoof's Capital Bikeshare Project - Modeling Data Cleaning Notebook

#### Developed by Clare Cruz

## Package Import

In [3]:
import pandas as pd
import datetime 
import numpy as np

pd.set_option("display.max_rows", 10)

## Data Import

We are bringing in the data all the months in 2020 but we've included December 2018 to determine the starting number of bikes. <br>
Please note that it is **required for the data files to be downloaded onto your computer in order for the code to run** <br>
These dates were chosen to avoid complications from COVID <br>
We are also bringing in the station capacity data from https://opendata.dc.gov/ which provides data as of the pull date, which was 10/25/2021. <br>
We have also included daily weather data from https://www.ncei.noaa.gov/. This weather is taken from the Washington Dulles airport to represent the general DC area.

In [7]:
bike_capacity = pd.read_csv('Data/Capital_Bike_Share_Locations.csv')
last_year_bike_data = pd.read_csv('Data/201812-capitalbikeshare-tripdata.csv')
weather_data = pd.read_csv('Data/weather_data_dc_2019.csv')

In [3]:
all_filenames = ['Data/201901-capitalbikeshare-tripdata.csv',
                 'Data/201902-capitalbikeshare-tripdata.csv',
                 'Data/201903-capitalbikeshare-tripdata.csv',
                 'Data/201904-capitalbikeshare-tripdata.csv',
                 'Data/201905-capitalbikeshare-tripdata.csv',
                 'Data/201906-capitalbikeshare-tripdata.csv',
                 'Data/201908-capitalbikeshare-tripdata.csv',
                 'Data/201909-capitalbikeshare-tripdata.csv',
                 'Data/201910-capitalbikeshare-tripdata.csv',
                 'Data/201911-capitalbikeshare-tripdata.csv',
                 'Data/201912-capitalbikeshare-tripdata.csv']

For some god foresaken reason, the date time formatting in the July 2019 data is not the same as the other months, so we will fix that real quick

In [4]:
july_df = pd.read_csv('Data/201907-capitalbikeshare-tripdata.csv')
july_df['Start date'] = pd.to_datetime(july_df['Start date']).dt.strftime('%Y-%m-%d %H:%M:%S')
july_df['End date'] = pd.to_datetime(july_df['End date']).dt.strftime('%Y-%m-%d %H:%M:%S')
july_df.head()

,Duration,Start date,End date,Start station number,Start station,End station number,End station,Bike number,Member type
0,528,2019-07-01 00:00:00,2019-07-01 00:08:00,31312,Wisconsin Ave & O St NW,31234,20th & O St NW / Dupont South,W23055,Member
1,548,2019-07-01 00:00:00,2019-07-01 00:09:00,31321,15th St & Constitution Ave NW,31270,8th & D St NW,W20124,Member
2,1612,2019-07-01 00:00:00,2019-07-01 00:27:00,31211,Kennedy Center,31289,Henry Bacon Dr & Lincoln Memorial Circle NW,W00461,Member
3,534,2019-07-01 00:00:00,2019-07-01 00:09:00,31321,15th St & Constitution Ave NW,31270,8th & D St NW,W23106,Member
4,530,2019-07-01 00:00:00,2019-07-01 00:09:00,31201,15th & P St NW,31109,7th & T St NW,W22710,Casual


In [5]:
combined_data = pd.concat([pd.read_csv(f) for f in all_filenames ])
combined_data = pd.concat([combined_data,july_df])
combined_data.shape

(3398417, 9)

In [6]:
# Checking to see if June was brought in
combined_data.head()

,Duration,Start date,End date,Start station number,Start station,End station number,End station,Bike number,Member type
0,230,2019-01-01 00:04:48,2019-01-01 00:08:39,31203,14th & Rhode Island Ave NW,31200,Massachusetts Ave & Dupont Circle NW,E00141,Member
1,1549,2019-01-01 00:06:37,2019-01-01 00:32:27,31321,15th St & Constitution Ave NW,31114,18th St & Wyoming Ave NW,W24067,Casual
2,177,2019-01-01 00:08:46,2019-01-01 00:11:44,31104,Adams Mill & Columbia Rd NW,31323,Woodley Park Metro / Calvert St & Connecticut ...,W22654,Casual
3,228,2019-01-01 00:08:47,2019-01-01 00:12:35,31281,8th & O St NW,31280,11th & S St NW,W22336,Member
4,1300,2019-01-01 00:12:29,2019-01-01 00:34:10,31014,Lynn & 19th St North,31923,Columbia Pike & S Taylor St,70004,Member


In [7]:
# Checking to make sure December was brought in
combined_data.tail()

,Duration,Start date,End date,Start station number,Start station,End station number,End station,Bike number,Member type
356640,1596,2019-07-31 23:55:00,2019-08-01 00:22:00,31249,Jefferson Memorial,31290,17th St & Independence Ave SW,78598,Member
356641,588,2019-07-31 23:56:00,2019-08-01 00:05:00,31104,Adams Mill & Columbia Rd NW,31128,23rd & M St NW,W24040,Member
356642,737,2019-07-31 23:57:00,2019-08-01 00:10:00,31115,Columbia Rd & Georgia Ave NW,31267,17th St & Massachusetts Ave NW,W23142,Member
356643,359,2019-07-31 23:57:00,2019-08-01 00:03:00,31117,15th & Euclid St NW,31282,16th & R St NW,W22184,Member
356644,513,2019-07-31 23:59:00,2019-08-01 00:08:00,31516,Rhode Island Ave Metro,31510,18th St & Rhode Island Ave NE,W22855,Member


#### Datatime formatting

To make life easier, we will format the date data to Python's datetime type

Please note that for now, we will treat the last_year data and the remaining months data separately so that we can separately calculate the starting number of bikes. 

In [8]:
# Transform the data to pandas date time variables
last_year_bike_data['Start date'] = pd.to_datetime(last_year_bike_data['Start date'], format = '%Y-%m-%d %H:%M:%S')
last_year_bike_data['End date'] = pd.to_datetime(last_year_bike_data['End date'], format = '%Y-%m-%d %H:%M:%S')

combined_data['Start date'] = pd.to_datetime(combined_data['Start date'], format = '%Y-%m-%d %H:%M:%S')
combined_data['End date'] = pd.to_datetime(combined_data['End date'], format = '%Y-%m-%d %H:%M:%S')

#### Extract the day info and time info separately

In [9]:
# Extract the start and end times and date into separate columns
last_year_bike_data['Start_time'] = last_year_bike_data['Start date'].dt.time
last_year_bike_data['End_time'] = last_year_bike_data['End date'].dt.time

last_year_bike_data['Start_day'] = last_year_bike_data['Start date'].dt.date
last_year_bike_data['End_day'] = last_year_bike_data['End date'].dt.date

combined_data['Start_time'] = combined_data['Start date'].dt.time
combined_data['End_time'] = combined_data['End date'].dt.time

combined_data['Start_day'] = combined_data['Start date'].dt.date
combined_data['End_day'] = combined_data['End date'].dt.date

In [10]:
# Define hour 
last_year_bike_data['start_hour'] = last_year_bike_data['Start date'].dt.hour
combined_data['start_hour'] = combined_data['Start date'].dt.hour

### Accounting for the reshuffling

One of the issues in this project is the van reshuffling. Our solution for this is to add rows to the data to account for these transitions. The outline of the reshuffling looks like: <br>
For one bike: <br>
Trip 1 - Station A --> Station B <br>
Trip 2 - Station B --> Van (**Add row**) <br>
Trip 3 - Van --> Station C (**Add row**) <br>
Trip 4 - Station C --> Station D

In [11]:
# Drop the rows that have a blank bike number (there are a few)
sorted_bike_data = combined_data[combined_data['Bike number'].notna()]
shuffle_bike_data = sorted_bike_data.sort_values(['Bike number', 'Start date'])
shuffle_bike_data
# Reset the index so that our indices work in the for loop in the next chunk
shuffle_bike_data.reset_index(inplace = True, drop = True)

In [12]:
# Adds two rows each time the van has reshuffled the data
# A --> B
# B --> Van
# Van --> C
# C -->
count = 0
# For every row in the dataset
shuffle_bike_data_clean = []
for i in range(len(shuffle_bike_data)-1):
    if shuffle_bike_data.loc[i,'End station number'] != shuffle_bike_data.loc[i+1,'Start station number']:
        if shuffle_bike_data.loc[i,'Bike number'] == shuffle_bike_data.loc[i+1,'Bike number']:
            count += 1
            # Add the two rows to the dataframe
            shuffle_bike_data_clean.append([0, 
                                            shuffle_bike_data.loc[i,'End date'], 
                                            shuffle_bike_data.loc[i,'End date'], 
                                            shuffle_bike_data.loc[i,'End station number'],
                                            shuffle_bike_data.loc[i,'End station'], 
                                            'VAN001', 
                                            'VAN', 
                                            shuffle_bike_data.loc[i,'Bike number'],
                                            'NA',
                                            shuffle_bike_data.loc[i,'End_time'],
                                            shuffle_bike_data.loc[i,'End_time'],
                                            shuffle_bike_data.loc[i,'End_day'],
                                            shuffle_bike_data.loc[i,'End_day'], 
                                            shuffle_bike_data.loc[i,'start_hour']])
            shuffle_bike_data_clean.append([0,
                                            shuffle_bike_data.loc[i,'End date'],
                                            shuffle_bike_data.loc[i,'End date'],
                                            'VAN001',
                                            'VAN',
                                            shuffle_bike_data.loc[i+1,'Start station number'],
                                            shuffle_bike_data.loc[i+1,'Start station'],
                                            shuffle_bike_data.loc[i,'Bike number'],
                                            'NA',
                                            shuffle_bike_data.loc[i,'End_time'],
                                            shuffle_bike_data.loc[i,'End_time'], 
                                            shuffle_bike_data.loc[i,'End_day'],
                                            shuffle_bike_data.loc[i,'End_day'], 
                                            shuffle_bike_data.loc[i,'start_hour']])

In [13]:
# Make the new rows a dataframe
van_data = pd.DataFrame(shuffle_bike_data_clean, columns=['Duration', 'Start date', 'End date', 'Start station number', 'Start station', 'End station number', 'End station', 'Bike number', 'Member type','Start_time', 'End_time', 'Start_day', 'End_day', 'start_hour'])
van_data.shape

(697388, 14)

In [14]:
# Drop the rows that have a blank bike number (there are a few)
last_year_sorted_bike_data = last_year_bike_data[last_year_bike_data['Bike number'].notna()]
last_year_shuffle_bike_data = last_year_sorted_bike_data.sort_values(['Bike number', 'Start date'])
#shuffle_bike_data
# Reset the index so that our indices work in the for loop in the next chunk
last_year_shuffle_bike_data.reset_index(inplace = True, drop = True)

In [15]:
# Adds two rows each time the van has reshuffled the data
# A --> B
# B --> Van
# Van --> C
# C -->
count = 0
# For every row in the dataset
shuffle_bike_data_clean = []
for i in range(len(last_year_shuffle_bike_data)-1):
    if last_year_shuffle_bike_data.loc[i,'End station number'] != last_year_shuffle_bike_data.loc[i+1,'Start station number']:
        if last_year_shuffle_bike_data.loc[i,'Bike number'] == last_year_shuffle_bike_data.loc[i+1,'Bike number']:
            count += 1
            # Add the two rows to the dataframe
            shuffle_bike_data_clean.append([0, 
                                            last_year_shuffle_bike_data.loc[i,'End date'], 
                                            last_year_shuffle_bike_data.loc[i,'End date'], 
                                            last_year_shuffle_bike_data.loc[i,'End station number'],
                                            last_year_shuffle_bike_data.loc[i,'End station'], 
                                            'VAN001', 
                                            'VAN', 
                                            last_year_shuffle_bike_data.loc[i,'Bike number'],
                                            'NA',
                                            last_year_shuffle_bike_data.loc[i,'End_time'],
                                            last_year_shuffle_bike_data.loc[i,'End_time'],
                                            last_year_shuffle_bike_data.loc[i,'End_day'],
                                            last_year_shuffle_bike_data.loc[i,'End_day'], 
                                            last_year_shuffle_bike_data.loc[i,'start_hour']])
            shuffle_bike_data_clean.append([0,
                                            last_year_shuffle_bike_data.loc[i,'End date'],
                                            last_year_shuffle_bike_data.loc[i,'End date'],
                                            'VAN001',
                                            'VAN',
                                            last_year_shuffle_bike_data.loc[i+1,'Start station number'],
                                            last_year_shuffle_bike_data.loc[i+1,'Start station'],
                                            last_year_shuffle_bike_data.loc[i,'Bike number'],
                                            'NA',
                                            last_year_shuffle_bike_data.loc[i,'End_time'],
                                            last_year_shuffle_bike_data.loc[i,'End_time'], 
                                            last_year_shuffle_bike_data.loc[i,'End_day'],
                                            last_year_shuffle_bike_data.loc[i,'End_day'], 
                                            last_year_shuffle_bike_data.loc[i,'start_hour']])

In [16]:
# Make the new rows a dataframe
last_year_van_data = pd.DataFrame(shuffle_bike_data_clean, columns=['Duration', 'Start date', 'End date', 'Start station number', 'Start station', 'End station number', 'End station', 'Bike number', 'Member type','Start_time', 'End_time', 'Start_day', 'End_day', 'start_hour'])
last_year_van_data.shape

(36254, 14)

In [17]:
# Combine the van data
print(shuffle_bike_data.shape)
combined_shuffle_data = pd.concat([shuffle_bike_data, van_data], axis = 0)
combined_shuffle_data.shape

(3398403, 14)


(4095791, 14)

Data has been successfully added

In [18]:
# Combine the van data
print(last_year_shuffle_bike_data.shape)
last_year_shuffle_data = pd.concat([last_year_shuffle_bike_data, last_year_van_data], axis = 0)
last_year_shuffle_data.shape

(158853, 14)


(195107, 14)

Data has been succesfully added

#### Partition data into two tables: departures and arrivals

Calculating the number of arrivals and departures separately makes life easier.

In [19]:
# Make a ride id column so that we can merge two dfs together later
last_year_bike_data['ride_id'] = np.arange(last_year_bike_data.shape[0])
combined_shuffle_data['ride_id'] = np.arange(combined_shuffle_data.shape[0])

In [20]:
# Make two separate tables so we can easily calculate the number of arrivals/departures
last_year_arrivals = last_year_bike_data
last_year_departures = last_year_bike_data

combined_arrivals = combined_shuffle_data
combined_departures = combined_shuffle_data

#### Arrivals Table

In [21]:
# Aggregate the data
last_year_arrivals_agg = last_year_arrivals.groupby(['Start_day', 'start_hour', 'End station number']).count()
combined_arrivals_agg = combined_arrivals.groupby(['Start_day', 'start_hour', 'End station number']).count()

# We only need one column to get the counts 
last_year_arrivals_df = last_year_arrivals_agg['Bike number']
combined_arrivals_df = combined_arrivals_agg['Bike number']

# Move the start day, start hour, and station number variables to columns instead of indexes
last_year_arrivals_df = last_year_arrivals_df.reset_index(level = [0,1,2])
combined_arrivals_df = combined_arrivals_df.reset_index(level = [0,1,2])

# Normalize the variable names so we can merge tables
last_year_arrivals_df.rename({'End station number':'station_id', 'Bike number':'num_arrivals'}, axis = 1, inplace = True)
combined_arrivals_df.rename({'End station number':'station_id', 'Bike number':'num_arrivals'}, axis = 1, inplace = True)
combined_arrivals_df[combined_arrivals_df['station_id'] == 31000].head()

,Start_day,start_hour,station_id,num_arrivals
3452,2019-01-02,11,31000,1
4554,2019-01-02,17,31000,1
7840,2019-01-03,17,31000,1
10843,2019-01-04,15,31000,1
14052,2019-01-05,17,31000,1


#### Departures Table

In [22]:
# Aggregate the data
last_year_depart_agg = last_year_departures.groupby(['Start_day', 'start_hour','Start station number']).count()
combined_depart_agg = combined_departures.groupby(['Start_day', 'start_hour','Start station number']).count()

# We only need one column to get the counts 
last_year_depart_df = last_year_depart_agg['Bike number']
combined_depart_df = combined_depart_agg['Bike number']

# Move the start day, start hour, and station number variables to columns instead of indexes
last_year_depart_df = last_year_depart_df.reset_index(level = [0,1,2])
combined_depart_df = combined_depart_df.reset_index(level = [0,1,2])

# Normalize the variable names so we can merge tables
last_year_depart_df.rename({'Start station number':'station_id', 'Bike number':'num_depart'}, axis = 1, inplace = True)
combined_depart_df.rename({'Start station number':'station_id', 'Bike number':'num_depart'}, axis = 1, inplace = True)

#### Merge departure and arrivals table together

In [23]:
last_year_final_bike_data = pd.merge(last_year_arrivals_df, last_year_depart_df, how = 'outer', on = ['Start_day', 'start_hour','station_id']).fillna(0)
combined_final_bike_data = pd.merge(combined_arrivals_df, combined_depart_df, how = 'outer', on = ['Start_day', 'start_hour','station_id']).fillna(0)

last_year_final_bike_data.reset_index(inplace = True)

In [24]:
combined_final_bike_data.head()

,Start_day,start_hour,station_id,num_arrivals,num_depart
0,2019-01-01,0,31030,1.0,0.0
1,2019-01-01,0,31032,1.0,0.0
2,2019-01-01,0,31041,1.0,1.0
3,2019-01-01,0,31048,1.0,0.0
4,2019-01-01,0,31055,1.0,0.0


### Calculating the starting number of bikes at every station

Another quirk to this dataset is the starting number of bikes. If we start out in June without looking at some relative history, we are assuming that all of this stations have no bikes to start with. Therefore, we will look at last_year data to get a sense of how many bikes the stations start out with. We considered a whole month because some bikes don't get used very often so we need to expand the timeframe to capture all the bikes.

The starting number of bikes is calculated by finding the last location for each bike. A high level outline of the code is: <br>
<br>
For every bike: <br>
>    For every station:
>>        When was the last time the bike arrived and departed there
>>        If the departure time is before the arrival time
>>>            Then that bike must be at that station and flag it

In [25]:
last_year_shuffle_data.reset_index(drop = True, inplace = True)

In [26]:
# Get the latest time every bike was at every station
start_depart = last_year_shuffle_data[['Bike number', 'Start date', 'Start station number']].loc[last_year_shuffle_data.groupby(['Start station number','Bike number'])['Start date'].idxmax()]
start_arr = last_year_shuffle_data[['Bike number', 'End date', 'End station number']].loc[last_year_shuffle_data.groupby(['End station number','Bike number'])['End date'].idxmax()]

start_arr.reset_index(inplace = True, drop = True)
start_depart.reset_index(inplace = True, drop = True)

start_arr.rename({'End date':'max_arrived', 'End station number':'station_id'},axis = 1, inplace = True)
start_depart.rename({'Start date':'max_depart', 'Start station number':'station_id'},axis = 1, inplace = True)

There are bikes that are present in last_year but not in the combined data so we need to exclude them

In [27]:
diff_bike = len(list(set(last_year_shuffle_data['Bike number'].value_counts().index)-set(combined_shuffle_data['Bike number'].value_counts().index)))
print('Number of bikes that are in last_year but not combined: {}'.format(diff_bike))

Number of bikes that are in May but not combined: 19


This is a very small number of bikes so we can disregard these bikes to repairs, stolen, or broken.

In [28]:
# Excluse any bikes that are not present in combined
start_arr_filtered = start_arr.loc[start_arr['Bike number'].isin(combined_data['Bike number'].value_counts().index)]
start_depart_filtered = start_depart.loc[start_depart['Bike number'].isin(combined_data['Bike number'].value_counts().index)]

In [29]:
# Merge the arrivals and departure together and create a flag that indicates the last station every bike is at
last_year_last_bike_full = pd.merge(start_arr_filtered,start_depart_filtered,  how = 'outer', left_on = ['Bike number','station_id'], right_on = ['Bike number','station_id'])
last_year_last_bike_full['flag'] = np.where(((last_year_last_bike_full.max_arrived > last_year_last_bike_full.max_depart) | (last_year_last_bike_full.max_depart.isna())) , 1, 0)
print('Number of bikes in last_year: {}'.format(len(last_year_shuffle_data['Bike number'].value_counts())))
print('Number of bikes in last_year after excluding bikes that are not present in combined: {}'.format(last_year_last_bike_full[last_year_last_bike_full['flag'] == 1]['flag'].sum()))

Number of bikes in May: 3982
Number of bikes in May after excluding bikes that are not present in combined: 3963


In [30]:
# Create a dataframe that matches the combined data so that the tables can be combined
last_year_last_bike = last_year_last_bike_full.groupby('station_id')['flag'].sum()
last_year_last_bike = pd.DataFrame(last_year_last_bike)
last_year_last_bike.reset_index(inplace = True)
last_year_last_bike['Start_day'] = datetime.date(day = 31, month = 12, year = 2018) # Can be anything
last_year_last_bike['start_hour'] = 23 # Can be anything
last_year_last_bike['num_depart'] = 0 # So we can get the starting number
last_year_last_bike.rename({'flag':'num_arrivals'}, axis = 1,inplace = True)
last_year_last_bike['diff'] = last_year_last_bike['num_arrivals']
last_year_last_bike.head()

,station_id,num_arrivals,Start_day,start_hour,num_depart,diff
0,31000,5,2018-12-31,23,0,5
1,31001,4,2018-12-31,23,0,4
2,31002,16,2018-12-31,23,0,16
3,31003,5,2018-12-31,23,0,5
4,31004,7,2018-12-31,23,0,7


#### Calculate the difference in arrivals and departures

In [31]:
combined_final_bike_data['diff'] = combined_final_bike_data['num_arrivals'] - combined_final_bike_data['num_depart']

In [32]:
full_data = pd.concat([last_year_last_bike, combined_final_bike_data], axis = 0)
full_data.head()

,station_id,num_arrivals,Start_day,start_hour,num_depart,diff
0,31000,5.0,2018-12-31,23,0.0,5.0
1,31001,4.0,2018-12-31,23,0.0,4.0
2,31002,16.0,2018-12-31,23,0.0,16.0
3,31003,5.0,2018-12-31,23,0.0,5.0
4,31004,7.0,2018-12-31,23,0.0,7.0


### Bikeshare capacity data

Now we need the capacity information from the other dataset so that we can set a limit to the cumulative difference for bike availability

In [33]:
filtered_capacity = bike_capacity[['NAME','NUM_DOCKS_AVAILABLE','NUM_BIKES_AVAILABLE' ,'LONGITUDE', "LATITUDE", "REGION_NAME"]]
filtered_capacity.head()

,NAME,NUM_DOCKS_AVAILABLE,NUM_BIKES_AVAILABLE,LONGITUDE,LATITUDE,REGION_NAME
0,Duke St & John Carlyle St,5,10,-77.060866,38.804378,"Alexandria, VA"
1,Langston Blvd & N Cleveland St,8,7,-77.091690,38.894941,"Arlington, VA"
2,Arlington Blvd & S George Mason Dr,12,3,-77.104503,38.869442,"Arlington, VA"
3,10th & G St NW,15,7,-77.026235,38.898243,"Washington, DC"
4,Langston Blvd & N Scott St,4,11,-77.080851,38.897612,"Arlington, VA"


In [34]:
# Get a dataframe with all the bike stations and their names
unique_stations = combined_data.drop_duplicates(subset=['Start station number'])[['Start station number', 'Start station']]

In [114]:
# Add back the station names and the capacity to the dataset
full_data_stations = pd.merge(full_data, unique_stations, how = 'left', left_on = 'station_id', right_on = 'Start station number')
full_bike_data = pd.merge(full_data_stations, filtered_capacity, how = 'left', left_on = 'Start station', right_on = 'NAME')

In [115]:
# Remove all the stations that are not in our dataset
bad_stations = full_bike_data[full_bike_data['NAME'].isna()]['station_id'].value_counts().index.tolist()
bad_stations.remove('VAN001') # don't want to delete the van data
#print(bad_stations)
print('There are {} stations that do not have a capacity'.format(len(bad_stations)))

#Filter the dataset to the stations that have a capacity
full_bike_data = full_bike_data[~full_bike_data['station_id'].isin(bad_stations)]
full_bike_data.shape

There are 94 stations that do not have a capacity


(1552190, 14)

There's only 30 stations that do not have a capacity and all have either been moved, removed, or renamed

In [116]:
# Calculate the total capacity
full_bike_data['total_capacity'] = full_bike_data['NUM_DOCKS_AVAILABLE'] + full_bike_data['NUM_BIKES_AVAILABLE']

In [117]:
# Drop the redundant info 
full_bike_data.drop(columns = ['NAME', 'Start station number','NUM_DOCKS_AVAILABLE','NUM_BIKES_AVAILABLE'], inplace = True)
full_bike_data.rename({'Start station': 'station_name',  
                       'LONGITUDE': 'longitude', 
                       'REGION_NAME': 'region_name',
                       'LATITUDE':'latitude'} , axis = 1, inplace = True)

There is a weird random station that is marked at station 0 which should be something else so we will fix that right here

In [118]:
full_bike_data.loc[full_bike_data.station_id == 0, 'station_id'] = 31227

In [119]:
full_bike_data.head()

,station_id,num_arrivals,Start_day,start_hour,num_depart,diff,station_name,longitude,latitude,region_name,total_capacity
0,31000,5.0,2018-12-31,23,0.0,5.0,Eads St & 15th St S,-77.053230,38.858971,"Arlington, VA",14.0
2,31002,16.0,2018-12-31,23,0.0,16.0,Crystal Dr & 20th St S,-77.049232,38.856425,"Arlington, VA",17.0
3,31003,5.0,2018-12-31,23,0.0,5.0,Crystal Dr & 15th St S,-77.049417,38.861056,"Arlington, VA",15.0
9,31009,9.0,2018-12-31,23,0.0,9.0,Crystal Dr & 27th St S,-77.051514,38.848466,"Arlington, VA",10.0
11,31011,13.0,2018-12-31,23,0.0,13.0,Crystal Dr & 23rd St S,-77.049723,38.852929,"Arlington, VA",17.0


### Determine the starting number of bikes for our timeframe

#### Calculate cumulative sum of diff for every station

In [120]:
# Sort the values so that we can view the stations individually one at a time in chronological order
full_bike_data.sort_values(['station_id','Start_day', 'start_hour'], inplace = True)
full_bike_data.reset_index(drop = True, inplace = True)

In [121]:
full_bike_data['cumu_sum1'] = full_bike_data[['station_id','Start_day','start_hour','diff']].groupby(['station_id','Start_day','start_hour']).sum().groupby(level=0).cumsum().reset_index()['diff']

There is a maximum and minimum limit on the cumulative sum to account for those outlying changes in bike availability that would make the cumulative sum negative or much higher than the station can hold. To measure the number of times this happens, the number of times the cumulative sum goes over a station's capacity or goes negative is measured.

In [122]:
#Some of the numbers are still a little weird - so we will use the custom cumulative sum function
station_num = full_bike_data.loc[0,'station_id']
cumu_sum = 0
zero_counter = 0
cap_counter = 0
for i in range(0,len(full_bike_data)):
    # Check to see if the station number has changed
    if full_bike_data.loc[i,'station_id'] != station_num:
        # If it has, reset the sum to the new station
        station_num = full_bike_data.loc[i,'station_id']
        cumu_sum = 0
    # If the cumu sum has reached max capacity
    if cumu_sum + full_bike_data.loc[i, 'diff'] >= full_bike_data.loc[i,'total_capacity']:
        full_bike_data.at[i,'cumu_sum2'] = full_bike_data.loc[i,'total_capacity']
        cumu_sum = full_bike_data.loc[i,'total_capacity']
        cap_counter +=1
    # If the cumu sum has reached minimum capacity
    elif cumu_sum + full_bike_data.loc[i, 'diff'] < 0:
        full_bike_data.at[i,'cumu_sum2'] = 0
        cumu_sum = 0
        zero_counter += 1
    else:
        # Add to the cumulative sum and add the data point to the table
        cumu_sum += full_bike_data.loc[i, 'diff'] 
        full_bike_data.at[i,'cumu_sum2'] = cumu_sum

### Bike availability error investigation

#### How representative are the two cumulative sum calculations?

In [123]:
print("Number of times restricted cumu sum is negative: {}".format(zero_counter))
print("Number of times restricted cumu sum is greater than capacity: {}".format(cap_counter))
print("Percent of bikes affected by restriction: {}".format((zero_counter + cap_counter)/full_bike_data.shape[0]*100))

Number of times restricted cumu sum is negative: 19066
Number of times restricted cumu sum is greater than capacity: 39763
Percent of bikes affected by restriction: 3.790064360677494


In [124]:
print("Number of times raw cumu sum is greater than capacity: {}".format(full_bike_data[abs(full_bike_data['cumu_sum1']) > full_bike_data['total_capacity']].shape[0]))
print("Number of times raw cumu sum is negative: {}".format(full_bike_data[abs(full_bike_data['cumu_sum1']) < 0].shape[0]))

Number of times raw cumu sum is greater than capacity: 316039
Number of times raw cumu sum is negative: 0


- It looks like we should focus on the restricted cumulative sum since it affects less data points
- The raw cumulative sum indicates that total capacity may not be correct all the time

#### Bad starting points

The main cause behind the errors in the cumulative sum is our inaccurate measure of the starting point
but this only affects **20 stations** in our dataset and the numbers are fairly close so we will assume this is
a natural error that can't be accounted for

In [125]:
num_bad_starting_points = full_bike_data[(full_bike_data['Start_day'] == datetime.date(month = 12, year = 2018, day = 31))&(full_bike_data['num_arrivals']>full_bike_data['total_capacity'])].shape[0]
print("Number of stations with a starting number that is above its capacity: {}".format(num_bad_starting_points))

Number of stations with a starting number that is above its capacity: 20


#### Popular Events

There also appeared to be random **large spikes and dips in arrivals and departures for some stations**, further investigation saw that these events were related to popular events like baseball games <br>
For example, the station near the MLB Nationals Stadium had peak arrivals and departures around the time of the game. <br>
This article shows how this is a common practice for the company, and we can assume that other large spikes are also due to large events <b>
https://www.wnyc.org/story/285378-nine-percent-of-all-d-c-bike-share-bikes-are-at-washington-nationals-stadium-for-the-playoff-game/.

In [126]:
full_bike_data[(full_bike_data['station_id'] == 31648) & (abs(full_bike_data['diff']) > full_bike_data['total_capacity'])]

,station_id,num_arrivals,Start_day,start_hour,num_depart,diff,station_name,longitude,latitude,region_name,total_capacity,cumu_sum1,cumu_sum2
1332864,31648,48.0,2019-04-06,14,5.0,43.0,Potomac Ave & Half St SW,-77.010815,38.869683,"Washington, DC",23.0,10.0,23.0
1332867,31648,6.0,2019-04-06,17,56.0,-50.0,Potomac Ave & Half St SW,-77.010815,38.869683,"Washington, DC",23.0,7.0,0.0
1332899,31648,40.0,2019-04-09,19,0.0,40.0,Potomac Ave & Half St SW,-77.010815,38.869683,"Washington, DC",23.0,15.0,23.0
1332902,31648,0.0,2019-04-09,22,46.0,-46.0,Potomac Ave & Half St SW,-77.010815,38.869683,"Washington, DC",23.0,14.0,0.0
1333028,31648,35.0,2019-04-21,15,2.0,33.0,Potomac Ave & Half St SW,-77.010815,38.869683,"Washington, DC",23.0,6.0,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1334464,31648,0.0,2019-08-21,22,46.0,-46.0,Potomac Ave & Half St SW,-77.010815,38.869683,"Washington, DC",23.0,4.0,0.0
1334832,31648,45.0,2019-09-22,19,2.0,43.0,Potomac Ave & Half St SW,-77.010815,38.869683,"Washington, DC",23.0,4.0,23.0
1334835,31648,0.0,2019-09-22,22,49.0,-49.0,Potomac Ave & Half St SW,-77.010815,38.869683,"Washington, DC",23.0,8.0,0.0
1335015,31648,60.0,2019-10-06,15,11.0,49.0,Potomac Ave & Half St SW,-77.010815,38.869683,"Washington, DC",23.0,6.0,23.0


### Adding additional information

In [127]:
full_bike_data.drop(columns = ['cumu_sum1'], axis = 1, inplace = True)
full_bike_data.rename({'cumu_sum2':'cumu_sum'}, axis = 1, inplace = True)

#### Weekend vs Weekday

In [128]:
full_bike_data['percent_full'] = full_bike_data['cumu_sum']/full_bike_data['total_capacity']
full_bike_data['weekday_num'] = pd.to_datetime(full_bike_data['Start_day'], errors = 'coerce').dt.weekday
#full_bike_data.head()
full_bike_data['weekend_flag'] = np.where((full_bike_data['weekday_num'] == 6) | (full_bike_data['weekday_num'] == 5), 1, 0)

#### Weather

The weather data contains daily precipitation and temperature data. It gets merged with our full data set by the date.

In [129]:
weather_data["DATE"] = pd.to_datetime(weather_data['DATE'])
weather_data = weather_data.drop(columns = ['STATION',"NAME"])
weather_data.head()

,DATE,PRCP,SNOW,TAVG,TMAX,TMIN
0,2018-12-31,0.53,0.0,37,47,30
1,2019-01-01,0.00,0.0,53,62,40
2,2019-01-02,0.00,0.0,43,45,37
3,2019-01-03,0.00,0.0,44,49,30
4,2019-01-04,0.03,0.0,36,46,28


In [130]:
# Also need to change start day to a datetime so that we can merge on date
full_bike_data['Start_day'] = pd.to_datetime(full_bike_data['Start_day'])

# Complete merge
full_bike_data = pd.merge(full_bike_data, weather_data, how = 'left', left_on = 'Start_day', right_on = "DATE")
full_bike_data.drop(columns = ['DATE'], inplace = True)
full_bike_data.head()

,station_id,num_arrivals,Start_day,start_hour,num_depart,diff,station_name,longitude,latitude,region_name,total_capacity,cumu_sum,percent_full,weekday_num,weekend_flag,PRCP,SNOW,TAVG,TMAX,TMIN
0,31000,5.0,2018-12-31,23,0.0,5.0,Eads St & 15th St S,-77.05323,38.858971,"Arlington, VA",14.0,5.0,0.357143,0,0,0.53,0.0,37,47,30
1,31000,0.0,2019-01-02,9,2.0,-2.0,Eads St & 15th St S,-77.05323,38.858971,"Arlington, VA",14.0,3.0,0.214286,2,0,0.00,0.0,43,45,37
2,31000,1.0,2019-01-02,11,0.0,1.0,Eads St & 15th St S,-77.05323,38.858971,"Arlington, VA",14.0,4.0,0.285714,2,0,0.00,0.0,43,45,37
3,31000,0.0,2019-01-02,12,1.0,-1.0,Eads St & 15th St S,-77.05323,38.858971,"Arlington, VA",14.0,3.0,0.214286,2,0,0.00,0.0,43,45,37
4,31000,1.0,2019-01-02,17,0.0,1.0,Eads St & 15th St S,-77.05323,38.858971,"Arlington, VA",14.0,4.0,0.285714,2,0,0.00,0.0,43,45,37


#### Holidays

In [131]:
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

cal = calendar()
holidays = cal.holidays(start= full_bike_data['Start_day'].min(), end=full_bike_data['Start_day'].max())
holidays
full_bike_data['holiday'] = pd.to_datetime(full_bike_data['Start_day']).isin(holidays)

#### Final data before export

In [1]:
full_bike_data.head()

NameError: name 'full_bike_data' is not defined

### Data Export

Change file path to desired location

In [133]:
#full_bike_data.to_csv('C:/Users/cbrig/OneDrive/CMU/Perspectives in Data Science/Group Project/cleaned_bike_data_2019.csv')